In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

In [19]:
# pip install yfinance pandas
import yfinance as yf
import pandas as pd
import math

def pct(x): return None if x is None else x * 100

def get_core_metrics_yf(ticker: str):
    t = yf.Ticker(ticker)

    # Prefer .get_info() (newer) then fallback to .info
    try:
        info = t.get_info()
    except Exception:
        info = getattr(t, "info", {}) or {}

    # Try annual financials for backup calculations
    fin = getattr(t, "financials", pd.DataFrame())
    revenue_growth = info.get("revenueGrowth")  # YoY (decimal)
    ebitda_margin = info.get("ebitdaMargins")   # decimal
    ebitda = info.get("ebitda")
    eps = info.get("trailingEps")
    eps_growth = info.get("earningsGrowth")     # YoY (decimal)
    pe = info.get("trailingPE")
    ev = info.get("enterpriseValue")

    # Fallbacks from statements if needed
    # EBITDA margin backup: EBITDA / Total Revenue (last fiscal year)
    if ebitda_margin is None and isinstance(fin, pd.DataFrame) and not fin.empty:
        try:
            col = fin.columns[0]  # most recent FY
            rev = fin.loc["Total Revenue", col]
            ebit = fin.loc["EBITDA", col]
            if rev and ebit:
                ebitda_margin = ebit / rev
            if ebitda is None:
                ebitda = ebit
        except Exception:
            pass

    # EV/EBITDA
    ev_ebitda = None
    if ev is not None and ebitda not in (None, 0) and not math.isclose(float(ebitda), 0.0):
        ev_ebitda = ev / ebitda

    # Last price (for context)
    try:
        price = t.history(period="1d")["Close"][-1]
    except Exception:
        price = None

    return {
        "ticker": ticker,
        "price": price,
        "Revenue Growth %": pct(revenue_growth),
        "EBITDA Margin %": pct(ebitda_margin),
        "EPS (ttm)": eps,
        "EPS Growth %": pct(eps_growth),
        "P/E (trailing)": pe,
        "EV/EBITDA": ev_ebitda,
    }

tickers = ["BRK", "QBTS", "NVDA", "SOFI", "TTD", "VST", "META", "IONQ", "AMZN", "AVGO", "GOOGL", "ACHR"]
rows = [get_core_metrics_yf(t) for t in tickers]
df = pd.DataFrame(rows)
print(df)

$BRK: possibly delisted; no price data found  (period=1d)
/var/folders/lx/ddlr7_8s19s00cjxkh6fqgyh0000gn/T/ipykernel_52201/1740150901.py:48: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  price = t.history(period="1d")["Close"][-1]
/var/folders/lx/ddlr7_8s19s00cjxkh6fqgyh0000gn/T/ipykernel_52201/1740150901.py:48: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  price = t.history(period="1d")["Close"][-1]
/var/folders/lx/ddlr7_8s19s00cjxkh6fqgyh0000gn/T/ipykernel_52201/1740150901.py:48: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as l

   ticker       price  Revenue Growth %  EBITDA Margin %  EPS (ttm)  \
0     BRK         NaN               NaN              NaN        NaN   
1    QBTS   15.960000              41.8         0.000000      -1.13   
2    NVDA  180.345001              69.2        59.419996       3.11   
3    SOFI   26.021000              43.9         0.000000       0.50   
4     TTD   55.400002              18.7        20.700000       0.83   
5     VST  196.990005              10.5        36.060002       6.23   
6    META  751.219971              21.6        52.728003      27.58   
7    IONQ   43.209999              81.8         0.000000      -2.06   
8    AMZN  231.639999              13.3        19.974001       6.55   
9    AVGO  308.779999              16.4        54.540000       2.73   
10  GOOGL  211.764999              13.8        37.922000       9.38   
11   ACHR    9.210000               NaN         0.000000      -1.27   

    EPS Growth %  P/E (trailing)  EV/EBITDA  
0            NaN             N

/var/folders/lx/ddlr7_8s19s00cjxkh6fqgyh0000gn/T/ipykernel_52201/1740150901.py:48: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  price = t.history(period="1d")["Close"][-1]
